<a href="https://colab.research.google.com/github/YaokunLin/ActuaryProgrammer/blob/main/VWMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import datetime
from datetime import datetime, timedelta, date
import requests
import json

# API documentation
# https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductcandles
'''
Historic rates for a product. Rates are returned in grouped buckets. 
Candle schema is of the form 
[timestamp, price_low, price_high, price_open, price_close]

max only 300 candles per requests 
'''
class VWAPCalculator(object):
  def __init__(self, pair = 'BTC-USD', endDate = None, windowSize = 200, dayDelta = 1):
    self._pair = pair
    self._apiUrl = 'https://api.pro.coinbase.com'
    self._windowSize = windowSize
    self._dayDelta = dayDelta
    endYr, endMonth, endDay = list(map(int, endDate.split('-'))) if endDate else [None, None, None]
    self._chosenDate = date(endYr, endMonth, endDay) if endDate else date.today()
    self._dateStart, self._dateEnd = self.getDates()
  
  def getDates(self):
    dateEnd = self._chosenDate
    dayDelta = timedelta(days = self._dayDelta) #intra-day when self._dayDelta = 1
    dateStart = dateEnd - (self._windowSize - 1) * dayDelta #subtract 1 to keep only 200 data points otherwise it would be 201 points
    return dateStart.isoformat(), dateEnd.isoformat()
  
  def refreshApi(self):
    parameters = {'start':self._dateStart,
                  'end':self._dateEnd,
                  'granularity': 86400}
    #The granularity field must be one of the following values: 
    #{60, 300, 900, 3600, 21600, 86400}. 
    #Otherwise, your request will be rejected. 
    #These values correspond to timeslices representing 
    #one minute, five minutes, fifteen minutes, one hour, six hours, and one day, respectively.
    response = requests.get(f"{self._apiUrl}/products/{self._pair}/candles",
                        params = parameters,
                        headers = {'content-type':'application/json'})
    
    self._response = response.json()
    '''
    Response Items
    Each bucket is an array of the following information:

    1. time bucket start time
    2. low lowest price during the bucket interval
    3. high highest price during the bucket interval
    4. open opening price (first trade) in the bucket interval
    5. close closing price (last trade) in the bucket interval
    6. volume volume of trading activity during the bucket interval
    '''

  def getVWAP(self):
    closeTimesVol=[close * vol for time, low, high, open, close, vol in self._response]
    sumVol = sum([vol for time, low, high, open, close, vol in self._response])
    if sumVol == 0: raise ZeroDivisionError("Data Not Avaliable")
    self._VWAP = sum(closeTimesVol)/sumVol
    return self._VWAP

  def __str__(self):
    dayDelta = timedelta(days = self._dayDelta)
    windwSize = len(self._response) 
    end = min(self._chosenDate, date.today())
    start = end - (windwSize - 1) * dayDelta
    return f"{self._pair}, Fr: {start}, To: {end}, windwSize: {windwSize}, VWAP: ${self._VWAP}"

  def updateDate(self, endDate):
    endYr, endMonth, endDay = list(map(int, endDate.split('-')))
    self._chosenDate = date(endYr, endMonth, endDay)
    self._dateStart, self._dateEnd = self.getDates()



In [ ]:
import time
print("Today's date:", date.today())

t0 = time.process_time()
btcUsdVwaps = VWAPCalculator() # dafault window is [today - 199 days, today]
btcUsdVwaps.refreshApi()
_ = btcUsdVwaps.getVWAP()
print(btcUsdVwaps)
t1 = time.process_time()
exeTime = t1 - t0
print('exe time:', exeTime)


print('----tests----')
testDates = sorted(['2021-01-01','2020-01-01','2022-01-01', '2021-11-11', '2015-10-15', '2015-07-20'])

for d in testDates:
  btcUsdVwaps.updateDate(d)
  btcUsdVwaps.refreshApi()
  _ = btcUsdVwaps.getVWAP()
  print('input date:', d, btcUsdVwaps)


Today's date: 2021-12-03
BTC-USD, Fr: 2021-05-18, To: 2021-12-03, windwSize: 200, VWAP: $44754.01102562989
exe time: 0.02506353799999994
----tests----
input date: 2015-07-20 BTC-USD, Fr: 2015-07-20, To: 2015-07-20, windwSize: 1, VWAP: $280.0
input date: 2015-10-15 BTC-USD, Fr: 2015-07-20, To: 2015-10-15, windwSize: 88, VWAP: $245.52869445853514
input date: 2020-01-01 BTC-USD, Fr: 2019-06-16, To: 2020-01-01, windwSize: 200, VWAP: $9798.874411759945
input date: 2021-01-01 BTC-USD, Fr: 2020-06-16, To: 2021-01-01, windwSize: 200, VWAP: $14998.043490869468
input date: 2021-11-11 BTC-USD, Fr: 2021-04-26, To: 2021-11-11, windwSize: 200, VWAP: $44589.599271790714
input date: 2022-01-01 BTC-USD, Fr: 2021-06-16, To: 2021-12-03, windwSize: 171, VWAP: $47272.50029049274


In [ ]:
d = '2020-11-22'
btcUsdVwaps.updateDate(d)
btcUsdVwaps.refreshApi()
_ = btcUsdVwaps.getVWAP()
print('input date:', d, btcUsdVwaps)

input date: 2020-11-22 BTC-USD, Fr: 2020-05-07, To: 2020-11-22, windwSize: 200, VWAP: $11346.784331163733


In [ ]:
print("Today's date:", date.today())

t0 = time.process_time()
ethUsdVwaps = VWAPCalculator(pair = 'ETH-USD') # dafault window is [today - 199 days, today]
ethUsdVwaps.refreshApi()
_ = ethUsdVwaps.getVWAP()
print(ethUsdVwaps)
t1 = time.process_time()
exeTime = t1 - t0
print('exe time:', exeTime)

print('----tests----')
testDates = sorted(['2021-01-01','2020-01-01','2022-01-01', '2021-11-11', '2016-09-30', '2016-05-18'])

for d in testDates:
  ethUsdVwaps.updateDate(d)
  ethUsdVwaps.refreshApi()
  _ = ethUsdVwaps.getVWAP()
  print('input date:', d, ethUsdVwaps)

Today's date: 2021-11-10
ETH-USD, Fr: 2021-04-25, To: 2021-11-10, windwSize: 200, VWAP: $2965.313186647746
exe time: 0.017569895999999918
----tests----
input date: 2016-05-18 ETH-USD, Fr: 2016-05-18, To: 2016-05-18, windwSize: 1, VWAP: $13.18
input date: 2016-09-30 ETH-USD, Fr: 2016-05-20, To: 2016-09-30, windwSize: 134, VWAP: $12.46493878221231
input date: 2020-01-01 ETH-USD, Fr: 2019-06-16, To: 2020-01-01, windwSize: 200, VWAP: $211.30496150602426
input date: 2021-01-01 ETH-USD, Fr: 2020-06-16, To: 2021-01-01, windwSize: 200, VWAP: $439.81925260179594
input date: 2021-11-11 ETH-USD, Fr: 2021-04-26, To: 2021-11-10, windwSize: 199, VWAP: $2968.4051628960283
input date: 2022-01-01 ETH-USD, Fr: 2021-06-16, To: 2021-11-10, windwSize: 148, VWAP: $3007.299070860193


In [ ]:
print("Today's date:", date.today())

t0 = time.process_time()
ethBtcVwaps = VWAPCalculator(pair = 'ETH-BTC') # dafault window is [today - 199 days, today]
ethBtcVwaps.refreshApi()
_ = ethBtcVwaps.getVWAP()
print(ethBtcVwaps)
t1 = time.process_time()
exeTime = t1 - t0
print('exe time:', exeTime)

print('----tests----')
testDates = sorted(['2021-01-01','2020-01-01','2022-01-01', '2021-11-11', '2016-09-30', '2016-05-18'])

for d in testDates:
  ethBtcVwaps.updateDate(d)
  ethBtcVwaps.refreshApi()
  _ = ethBtcVwaps.getVWAP()
  print('input date:', d, ethBtcVwaps)

Today's date: 2021-11-10
ETH-BTC, Fr: 2021-04-25, To: 2021-11-10, windwSize: 200, VWAP: $0.06623468059691555
exe time: 0.01720359600000032
----tests----
input date: 2016-05-18 ETH-BTC, Fr: 2016-05-18, To: 2016-05-18, windwSize: 1, VWAP: $0.0293
input date: 2016-09-30 ETH-BTC, Fr: 2016-05-19, To: 2016-09-30, windwSize: 135, VWAP: $0.02033232382213174
input date: 2020-01-01 ETH-BTC, Fr: 2019-06-16, To: 2020-01-01, windwSize: 200, VWAP: $0.02197601925178166
input date: 2021-01-01 ETH-BTC, Fr: 2020-06-16, To: 2021-01-01, windwSize: 200, VWAP: $0.030595392942664668
input date: 2021-11-11 ETH-BTC, Fr: 2021-04-26, To: 2021-11-10, windwSize: 199, VWAP: $0.06635134160797233
input date: 2022-01-01 ETH-BTC, Fr: 2021-06-16, To: 2021-11-10, windwSize: 148, VWAP: $0.06653489909662072


In [ ]:
import unittest

class TestVWAPCalculator(unittest.TestCase):

    def test_btcUsd1(self):
        '''Test case function for btcUsd'''
        self.calc = VWAPCalculator(windowSize = 1)
        self.calc.updateDate('2021-11-08')
        self.calc.refreshApi()
        result = self.calc.getVWAP()
        expected = 67554.84
        self.assertEqual(round(result, 5), round(expected, 5))

    def test_btcUsd2(self):
        '''Test case function for btcUsd'''
        self.calc = VWAPCalculator(windowSize = 2)
        self.calc.updateDate('2021-11-08')
        self.calc.refreshApi()
        result = self.calc.getVWAP()
        expected = (67554.84 * 17661.872972 + 63309.13 * 5596.024607) / (17661.872972 + 5596.024607)  
        self.assertEqual(round(result, 5), round(expected, 5))
  
    def test_ethUSD1(self):
        '''Test case function for ethUsd'''
        self.calc = VWAPCalculator(windowSize = 2, pair = 'ETH-USD')
        self.calc.updateDate('2021-11-08')
        self.calc.refreshApi()
        result = self.calc.getVWAP()
        expected = (4811.90 * 163884.293503 + 4617.02 * 85312.856137) / (163884.293503 + 85312.856137)  
        self.assertEqual(round(result, 5), round(expected, 5))

    def test_ethUSD2(self):
        '''Test case function for ethUsd'''
        self.calc = VWAPCalculator(windowSize = 1, pair = 'ETH-USD')
        self.calc.updateDate('2021-10-22')
        self.calc.refreshApi()
        result = self.calc.getVWAP()
        expected = 3971.55  
        self.assertEqual(round(result, 5), round(expected, 5))

    def test_ethBTC1(self):
        '''Test case function for ethUsd'''
        self.calc = VWAPCalculator(windowSize = 2, pair = 'ETH-BTC')
        self.calc.updateDate('2021-11-02')
        self.calc.refreshApi()
        result = self.calc.getVWAP()
        expected = (0.07262 * 36025.627982 + 0.07092 * 24140.364351) / (36025.627982 + 24140.364351)  
        self.assertEqual(round(result, 5), round(expected, 5))

    def test_ethBTC1(self):
        '''Test case function for ethUsd'''
        self.calc = VWAPCalculator(windowSize = 2, pair = 'ETH-BTC')
        self.calc.updateDate('2021-11-02')
        self.calc.refreshApi()
        result = self.calc.getVWAP()
        expected = (0.07262 * 36025.627982 + 0.07092 * 24140.364351) / (36025.627982 + 24140.364351)  
        self.assertEqual(round(result, 5), round(expected, 5))

    def test_ethBTC2(self):
        '''Test case function for ethUsd'''
        self.calc = VWAPCalculator(windowSize = 1, pair = 'ETH-BTC')
        self.calc.updateDate('2016-05-18')
        self.calc.refreshApi()
        result = self.calc.getVWAP()
        expected = 0.0293 
        self.assertEqual(round(result, 5), round(expected, 5))

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_btcUsd1 (__main__.TestVWAPCalculator)
Test case function for btcUsd ... ok
test_btcUsd2 (__main__.TestVWAPCalculator)
Test case function for btcUsd ... ok
test_ethBTC1 (__main__.TestVWAPCalculator)
Test case function for ethUsd ... ok
test_ethBTC2 (__main__.TestVWAPCalculator)
Test case function for ethUsd ... ok
test_ethUSD1 (__main__.TestVWAPCalculator)
Test case function for ethUsd ... ok
test_ethUSD2 (__main__.TestVWAPCalculator)
Test case function for ethUsd ... ok

----------------------------------------------------------------------
Ran 6 tests in 1.546s

OK
